# Análisis Seguimiento de Vehículos en Video

## Introducción y Descripción General
Este Jupyter Notebook proporciona un análisis detallado del código suministrado para el seguimiento de vehículos en un video. Utiliza el modelo YOLOv8 para la detección de objetos y SORT para el seguimiento de objetos. Es posible encontrar la implementación del algoritmo SORT en el archivo `./sort.py` disponible en este directorio. Este último es una versión modificada para esta aplicación y forma parte del desarrollo de  Alex Bewley para Queensland University of Technology y cuya investigación y documentación se encuentran disponibles en [el siguiente repositorio](https://github.com/abewley/sort?tab=readme-ov-file).

In [1]:
import numpy as np
import cv2
from ultralytics import YOLO
from sort import Sort


## Importación de Librerías Necesarias:
- `numpy`: Para operaciones matemáticas y manejo de arrays.
- `cv2` (OpenCV): Para el procesamiento de imágenes y video.
- `ultralytics.YOLOv8`: Un modelo pre-entrenado de detección de objetos.
- `sort.Sort`: Un algoritmo de seguimiento de objetos.

In [2]:

def resize_frame(frame, max_width=1280, max_height=720):
    h, w = frame.shape[:2]
    scale = min(max_width / w, max_height / h)
    new_w, new_h = int(w * scale), int(h * scale)
    return cv2.resize(frame, (new_w, new_h))


## Función `resize_frame`
Esta función redimensiona los frames del video manteniendo la relación de aspecto. Se utiliza para asegurar que el tamaño de los frames sea manejable para el procesamiento y visualización.

## Inicialización de Componentes Principales
El código inicializa componentes clave para la captura de video, detección de objetos y seguimiento:
1. **Captura de Video**: Se carga el video usando `cv2.VideoCapture`.
2. **Modelo YOLO**: Se inicializa el modelo YOLO para la detección de objetos en el video.
3. **Tracker SORT**: Se inicializa el tracker SORT para el seguimiento de objetos detectados por YOLO.

Es necesario entregar como base un video de prueba para `VideoCapture`. Además de un modelo pre-entrenado de YOLOv8. 

In [3]:
cap = cv2.VideoCapture("D:/Titulo/Github/vehicle_video_trajectory_extractor/videos/video_sim_30s_movement_estabilizado_filtrado.mp4")
model = YOLO("D:/Titulo/Github/vehicle_video_trajectory_extractor/models/cutom_dota.pt")
tracker = Sort()


## Ciclo Principal de Procesamiento de Video
El ciclo principal gestiona la lectura de cada frame, la detección de objetos, el seguimiento y la visualización:
- **Lectura de Frame**: Se lee cada frame del video.
- **Detección con YOLO**: Se detectan objetos en cada frame usando el modelo YOLO.
- **Filtrado y Seguimiento**: Se filtran las detecciones y se realiza el seguimiento de los vehículos.
- **Dibujar Resultados en Frame**: Se dibujan los identificadores y cuadros delimitadores en cada frame.

El siguiente ciclo recorre los cuadros del video de entrada realizando en cada uno los pasos descritos anteriormente.

In [16]:
cap = cv2.VideoCapture("D:/Titulo/Github/vehicle_video_trajectory_extractor/videos/video1_4k_30s_sift_estabilizado_filtrado.mp4")
model = YOLO("D:/Titulo/Github/vehicle_video_trajectory_extractor/models/cutom_dota.pt")
tracker = Sort(max_age=15)

while cap.isOpened():
    status, frame = cap.read() #lectura de frame
    if not status:
        break

    results = model(frame, stream=True, verbose=False) #deteccion de objetos con YOLO
    
    for res in results:
        #filtrado y seguimiento de objetos
        filtered_indices = np.where(res.boxes.conf.cpu().numpy() > 0.3)[0]
        data = res.boxes.data.cpu().numpy()[filtered_indices]
        tracks = tracker.update(data)
        new_data = []
        for i in range(len(tracks)):
            xmin, ymin, xmax, ymax, conf, class_id, track_id = tracks[i]
            xmin, ymin, xmax, ymax, track_id = int(xmin), int(ymin), int(xmax), int(ymax), int(track_id)
            class_name = res.names[int(class_id)]
            new_data.append([xmin, ymin, xmax, ymax, conf, class_name, track_id])
            #dibujado de resultados
            cv2.putText(img=frame, text=f"Id: {track_id}", org=(xmin, ymin-10), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=(0,255,0), thickness=2)
            cv2.rectangle(img=frame, pt1=(xmin, ymin), pt2=(xmax, ymax), color=(0, 255, 0), thickness=2)
    resized_frame = resize_frame(frame)
    cv2.imshow("frame", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


[       1313      1703.1      1342.9      1738.9      0.2508           1        2268]
[     20.892      936.75      49.657      956.45     0.26586           1        2263]
[     1119.7      376.04      1149.8      417.46     0.25486           1        2280]
[       2967      1032.1      3001.4        1051     0.27057           1        2275]
[     3014.4      439.35      3049.3      465.76     0.26528           1        2282]
[     2967.3      1031.6      3001.2      1050.1     0.32687           1        2275]
[       1313      1701.6      1346.3      1740.5     0.25586           1        2268]
[     2067.1      1039.5      2107.3      1059.3     0.79459           1        2262]
[     1448.6      1131.5      1487.6      1158.9     0.77401           1        2278]
[     1306.1      1744.1      1338.7      1776.4     0.31342           1        2271]
[     874.07      120.93      898.38      159.99     0.25985           1        2281]
[     1142.9      1301.2      1178.8        1330     0

# Análisis del Algoritmo SORT

El código proporcionado implementa el algoritmo SORT (Simple Online and Realtime Tracking), un método popular para el seguimiento de múltiples objetos en secuencias de video que, en combinación a un modelo de detección de objetos, entrega un arreglo de objetos con IDs únicos, algunos de sus componentes más importantes son:

## 1. Modelo de Filtro Kalman
El Filtro Kalman se emplea para predecir la posición futura de cada objeto en base a sus estados actuales y pasados. En el código, esto se maneja a través de la clase `KalmanBoxTracker`.

## 2. Asociación de Detecciones con Rastreadores
El algoritmo asocia las detecciones de objetos en cada frame del video con los rastreadores existentes. Esto se realiza mediante la función `associate_detections_to_trackers`, que utiliza el IoU (Intersección sobre la Unión) y un algoritmo de asignación lineal para encontrar la mejor correspondencia entre las detecciones actuales y los rastreadores existentes.

## 3. Manejo de la Vida Útil de los Rastreadores
Los rastreadores se actualizan o eliminan en función de su edad y el número de veces que han sido emparejados con detecciones. Esto permite que el algoritmo maneje situaciones donde los objetos pueden temporalmente desaparecer y reaparecer en el campo de visión.

 SORT rastrea objetos en videos al asociar detecciones de objetos en cada frame con rastreadores existentes utilizando el cálculo de IoU y actualiza estos rastreadores con información de movimiento utilizando filtros Kalman. Ahora, si bien su desarrollo es simple, es importante destacar que este cuenta con algunas limitaciones a estudiar, al presentar una baja fiabilidad cuando la detección presenta pérdidas importantes sobre el objeto que se busca seguir. 
